In [ ]:
import json
import time
import pickle
import numpy as np
import paho.mqtt.client as mqtt
from stable_baselines3 import SAC

# 📌 Parámetros de archivos (ajusta rutas si las cambias)
MODEL_PATH       = "test2_agent"
VECNORM_PATH     = "test2_agent_vecnormalize.plk"

# ————————————————————————————————————————
# 1) Carga del objeto VecNormalize guardado
# ————————————————————————————————————————
with open(VECNORM_PATH, "rb") as f:
    vec_norm = pickle.load(f)

# Durante la simulación no queremos actualizar medias ni recompensas
vec_norm.training    = False
vec_norm.norm_reward = False

# ————————————————————————————————————————
# 2) Carga del modelo SAC (sin env)  
# ————————————————————————————————————————
model = SAC.load(MODEL_PATH)
print(f"✅ Agent loaded from {MODEL_PATH}")

# ————————————————————————————————————————
# 3) MQTT: buffer y callbacks
# ————————————————————————————————————————
obs_buffer = {}

def on_connect(client, userdata, flags, rc):
    print(f"✅ Connected to MQTT broker (rc={rc})")
    client.subscribe("building/observation")
    print("✅ Subscribed to topic: building/observation")

def on_obs(client, userdata, msg):
    # Esperamos un JSON plano: {"sensor1": 1.23, "sensor2": 4.56, ...}
    obs_buffer.update(json.loads(msg.payload))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_obs
client.connect("mosquitto", 1883)
client.loop_start()

# ————————————————————————————————————————
# 4) Bucle principal: normalizar y predecir
# ————————————————————————————————————————
while True:
    if obs_buffer:
        # 4.1) Construir array ordenado
        keys = sorted(obs_buffer.keys())
        raw_obs = np.array([obs_buffer[k] for k in keys], dtype=np.float32)

        # 4.2) Normalizar usando VecNormalize
        #     VecNormalize.normalize_obs() aplica (obs - mean)/sqrt(var+eps) + clipping
        obs_norm = vec_norm.normalize_obs(raw_obs)

        # 4.3) Predecir acción
        action, _ = model.predict(obs_norm, deterministic=True)

        # 4.4) Publicar
        client.publish(
            "building/action",
            json.dumps(action.tolist())
        )

        obs_buffer.clear()

    time.sleep(0.001)


✅ Agent loaded from test2_agent
✅ Connected to MQTT broker (rc=0)
✅ Subscribed to topic: building/observation


/tmp/ipykernel_67887/1333720193.py:42: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()
